In [ ]:
# импорт библиотек
import pygame
import numpy as np

# инициализация Pygame
pygame.init()

# определение размеров окна и ячеек
width, height = 1000, 800
cell_size = 10  # Уменьшил размер ячейки для умещения большего количества клеток
rows, cols = height // cell_size, width // cell_size

# цвета
black = (0, 0, 0) # для живых клеток
white = (255, 255, 255) # для мертвых клеток
gray = (169, 169, 169) # для сетки

# создание поля
grid = np.zeros((rows, cols), dtype=int)
next_grid = np.copy(grid)

# создание окна
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Game of Life")

# шрифт
font = pygame.font.Font(None, 36)

# флаги для управления состоянием программы( это будет важно позже когда мы добавим кнопки и возможность задавать начальную позицию)
running = True
drawing = False
simulation_running = False

# функция для отрисовки поля
def draw_grid():
    # заполняем поле белым цветом
    screen.fill(white)
    # проверяем жива ли клетка
    for row in range(rows):
        for col in range(cols):
            # живая? отлично, закрашиваем черным цветом
            if grid[row, col] == 1:
                pygame.draw.rect(screen, black, (col * cell_size, row * cell_size, cell_size, cell_size))
            # сетка будет серой
            pygame.draw.rect(screen, gray, (col * cell_size, row * cell_size, cell_size, cell_size), 1)

# функция для обновления состояния поля
def up_grid():
    # копируем поле
    new_grid = np.copy(grid)
    # проходимся по полю
    for row in range(1, rows - 1):
        for col in range(1, cols - 1):
            # считаем соседей для ланной клетки
            neighbors = np.sum(grid[row - 1:row + 2, col - 1:col + 2]) - grid[row, col]
            # проверка для живой клетки
            if grid[row, col] == 1:
                # если у клетки меньше двух или больше трёх живых соседей- она умирает(
                if neighbors < 2 or neighbors > 3:
                    new_grid[row, col] = 0
            # проверка для живой мертвой
            else:
                # если у клетки 3 живых соседа- она оживает
                if neighbors == 3:
                    new_grid[row, col] = 1
    # возвращаем обновленное поле
    return new_grid

# основной цикл
while running:
    for event in pygame.event.get():
        # обработка событий
        # закрываем приложение- код останавливается
        if event.type == pygame.QUIT:
            running = False
        # нажимаем на ЛКМ- запуск процесса рисования(drawing), рисуем живые клеткм
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:
                drawing = True
        # отпускаем ЛКМ- процесс отрисовки останавливается
        elif event.type == pygame.MOUSEBUTTONUP:
            if event.button == 1:
                drawing = False
        # во время процесса отрисовки двигаем мышкой- закрашиваются соответствующие клетки
        elif event.type == pygame.MOUSEMOTION:
            if drawing and not simulation_running:
                x, y = event.pos
                col, row = x // cell_size, y // cell_size
                grid[row, col] = 1
        # отработка клавиш
        elif event.type == pygame.KEYDOWN:
            # пробел - стоп/запуск игры
            if event.key == pygame.K_SPACE:
                simulation_running = not simulation_running
            # c - очистить поле(все клетки умирают)
            elif event.key == pygame.K_c:
                grid = np.zeros((rows, cols), dtype=int)

    # отрисовка поля
    draw_grid()

    # если флаг "эволюции" равен True, то обновляем состояние поля
    if simulation_running and not drawing:
        # обновление состояния поля
        next_grid = up_grid()
        grid = np.copy(next_grid)

    # отрисовка кнопок(настраиваем цвет по RGB и позицию на экране)
    pygame.draw.rect(screen, white, (0, height - 50, width, 50)) # чисто фон для кнопок
    start_button = pygame.draw.rect(screen, (0, 255, 0), (50, height - 40, 80, 30)) # будущая кнопка Start
    finish_button = pygame.draw.rect(screen, (255, 0, 0), (150, height - 40, 80, 30)) # будушая кнопка Finish

    # отрисовка текста на кнопках
    start_text = font.render("Start", True, black)
    finish_text = font.render("Finish", True, black)

    # размещение текста на кнопках
    screen.blit(start_text, (start_button.centerx - start_text.get_width() // 2, start_button.centery - start_text.get_height() // 2))
    screen.blit(finish_text, (finish_button.centerx - finish_text.get_width() // 2, finish_button.centery - finish_text.get_height() // 2))

    # обработка событий кнопок
    # если жмем Start- флаг для обновления поля переходит в состояние True, и игра стартует
    if start_button.collidepoint(pygame.mouse.get_pos()) and pygame.mouse.get_pressed()[0]:
        simulation_running = True
    # если же жмем Finish- происходит противоположное и симуляция останавливается
    if finish_button.collidepoint(pygame.mouse.get_pos()) and pygame.mouse.get_pressed()[0]:
        simulation_running = False

    # обновление экрана
    pygame.display.flip()

# завершение программы
pygame.quit()